# Model on PCT_PRICE_CHANGE_DETRENDED (%) <= 0

## Step 0: Preparation: Load the processed content and original model

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import os

data = pd.read_csv("../10_Data_Clean/model_track_data.csv")

# Configuration
TARGET_VARIABLE = 'PCT_PRICE_CHANGE_DETRENDED (%)'
RAW_TARGET_VARIABLE = 'PCT_PRICE_CHANGE (%)'
YEAR_COLUMN = 'YEAR'
GEO_UNIQUE_ID_COLUMN = 'GEO_UNIQUE_ID'
TRAIN_YEAR_CUTOFF = 2020

# XGBoost specific params (can be tuned later)
XGB_PARAMS = {
    'objective': 'reg:squarederror', # Objective function for regression
    'n_estimators': 100,            # Number of boosting rounds/trees
    'learning_rate': 0.1,           # Step size shrinkage
    'max_depth': 5,                 # Maximum tree depth
    'subsample': 0.8,               # Fraction of samples used per tree
    'colsample_bytree': 0.8,        # Fraction of features used per tree
    'random_state': 42,
    'n_jobs': -1                    # Use all available CPU cores
}

# Step 1: Detrending Target Variable
print("--- Detrending Target Variable ---")
# Ensure YEAR is numeric
data[YEAR_COLUMN] = pd.to_numeric(data[YEAR_COLUMN], errors='coerce')
data = data.dropna(subset=[YEAR_COLUMN, RAW_TARGET_VARIABLE]) # Drop rows where year or target is missing

trend_model = LinearRegression()
# Reshape YEAR for sklearn compatibility
trend_model.fit(data[[YEAR_COLUMN]], data[RAW_TARGET_VARIABLE])
predicted_trend = trend_model.predict(data[[YEAR_COLUMN]])
data[TARGET_VARIABLE] = data[RAW_TARGET_VARIABLE] - predicted_trend
print(f"Target variable '{TARGET_VARIABLE}' created.")
print(f"Target variable mean: {data[TARGET_VARIABLE].mean():.4f}, std: {data[TARGET_VARIABLE].std():.4f}")




# Step 2: Define Feature Sets
print("\n--- Defining Feature Sets ---")
# Carefully select features, avoiding leakage from future/target variables
baseline_features = [
    'Median_Household_Income', 'Total_Population', 'Avg_Household_Size',
    'Gini_Index', 'Employment_Rate', 'Below_Poverty_Rate',
    'Rate_College_or_Higher', 'Black_Portion', 'White_Portion',
    'American_Indian_and_Alaska_Native_Portion', 'Asian_Portion',
    # Lagged Price Features
    'HOME_PRICE_LAG1', 'PRICE_CHANGE_LAG1', 'PRICE_CHANGE_DIFF',
    'ROLLING_1yr_PRICE_CHANGE', 'ROLLING_2yr_PRICE_CHANGE_STD'
    # Avoid interaction terms involving FIRE here
]

wildfire_features = [
    'NUM_FIRES', 'TOTAL_AREA_BURNED_IN_M2', 'AVG_FIRE_DURATION_DAYS',
    'MAX_PCT_TRACT_BURNED', 'ANY_MAJOR_FIRE', 'FIRE_EXPOSED',
    'PREV_MAX_PCT_TRACT_BURNED', 'FIRE_LAST_YEAR', 'FIRE_SHOCK',
    'YEARS_SINCE_LAST_FIRE', 'CUMULATIVE_AREA_BURNED_LAST_2YRS',
    # Interaction terms
    'FIRE_EXPOSED_x_PRICE_CHANGE_LAG1',
    'MAX_PCT_TRACT_BURNED_x_ROLLING_1yr_PRICE_CHANGE',
    'FIRE_EXPOSED_x_Median_Household_Income',
    'FIRE_EXPOSED_x_Below_Poverty_Rate'
]

# Ensure all defined features exist in the dataframe columns
baseline_features = [f for f in baseline_features if f in data.columns]
wildfire_features = [f for f in wildfire_features if f in data.columns]

# Combine for the second model
temp_all_features = list(set(baseline_features + wildfire_features)) 
all_features = sorted(temp_all_features)

# Verify no target/future leakage in feature lists
leaky_features = ['PRICE', 'NEXT_YEAR_PRICE', 'PRICE_CHANGE', RAW_TARGET_VARIABLE, TARGET_VARIABLE]






# Step 3: Sort Data and Temporal Split
print(f"\n--- Splitting Data: Train <= {TRAIN_YEAR_CUTOFF}, Test > {TRAIN_YEAR_CUTOFF} ---")
data = data.sort_values(by=[YEAR_COLUMN, GEO_UNIQUE_ID_COLUMN]).reset_index(drop=True)

train_data = data[data[YEAR_COLUMN] <= TRAIN_YEAR_CUTOFF].copy()
test_data = data[data[YEAR_COLUMN] > TRAIN_YEAR_CUTOFF].copy()


# Separate features (X) and target (y) for both sets
y_train = train_data[TARGET_VARIABLE]
y_test = test_data[TARGET_VARIABLE]

X_train_base = train_data[baseline_features]
X_test_base = test_data[baseline_features]

X_train_wf = train_data[all_features]
X_test_wf = test_data[all_features]
    
    



# Step 4: Imputation and Scaling
print("\n--- Applying Imputation and Scaling ---")

# --- Imputation ---
# Fit Imputer ONLY on training data, then transform both train and test
imputer_wf = SimpleImputer(strategy='median')
X_train_wf_imputed = imputer_wf.fit_transform(X_train_wf)
X_test_wf_imputed = imputer_wf.transform(X_test_wf)
print(f"Imputed NaNs in All features using median strategy.")

# --- Scaling ---
scaler_wf = StandardScaler()
X_train_wf_scaled = scaler_wf.fit_transform(X_train_wf_imputed)
X_test_wf_scaled = scaler_wf.transform(X_test_wf_imputed)
print(f"Scaled All features using StandardScaler.")

# Convert back to DataFrames (maintains column names for importance plots)
X_train_wf_scaled = pd.DataFrame(X_train_wf_scaled, columns=all_features, index=X_train_wf.index)
X_test_wf_scaled = pd.DataFrame(X_test_wf_scaled, columns=all_features, index=X_test_wf.index)

print("Imputation and Scaling complete.")




# Step 5: Model Training
print("\n--- Training Models ---")


rf_wf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10, min_samples_leaf=5)
rf_wf.fit(X_train_wf_scaled, y_train)
print("Trained Random Forest (Wildfire Features).")

# --- Original XGBoost ---
xgb_wf = XGBRegressor(**XGB_PARAMS)
xgb_wf.fit(X_train_wf_scaled, y_train)
print("Trained XGBoost (Wildfire Features).")


y_original_pred_xgb_wf = xgb_wf.predict(X_test_wf_scaled)
print("get the original model prediciton")

--- Detrending Target Variable ---
Target variable 'PCT_PRICE_CHANGE_DETRENDED (%)' created.
Target variable mean: 0.0000, std: 10.5283

--- Defining Feature Sets ---

--- Splitting Data: Train <= 2020, Test > 2020 ---

--- Applying Imputation and Scaling ---
Imputed NaNs in All features using median strategy.
Scaled All features using StandardScaler.
Imputation and Scaling complete.

--- Training Models ---
Trained Random Forest (Wildfire Features).
Trained XGBoost (Wildfire Features).
get the original model prediciton


## Step 1: Filter out samples with y<=0

In [6]:
# Define threshold
severe_threshold = 0

# Train set: only drop samples
drop_train_idx = (y_train < severe_threshold)
X_train_drop = X_train_wf_scaled[drop_train_idx]
y_train_drop = y_train[drop_train_idx]

# Test set: only drop samples
drop_test_idx = (y_test < severe_threshold)
X_test_drop = X_test_wf_scaled[drop_test_idx]
y_test_drop = y_test[drop_test_idx]

print(f"Drop Training Samples: {X_train_drop.shape[0]}")
print(f"Drop Testing Samples:  {X_test_drop.shape[0]}")


Drop Training Samples: 34639
Drop Testing Samples:  7085


## Step 2: Build a new regression model (XGBoost)

In [18]:
from lightgbm import LGBMRegressor

# Define XGBoost model
drop_model_xgb = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# # Define LightGBM model
# drop_model_lgbm = LGBMRegressor(
#     objective='regression',
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     n_jobs=-1
# )

# Train on y<0 subset
drop_model_xgb.fit(X_train_drop, y_train_drop)
# drop_model_lgbm.fit(X_train_drop, y_train_drop)

print("Trained Drop-Specific XGBoost Model.")
# print("Trained Drop-Specific LightGBM Model.")

y_pred_drop_xgb = drop_model_xgb.predict(X_test_drop)
print("Predicted Drop-Specific XGBoost Model.")
# Evaluate XGBoost
rmse_drop_xgb = np.sqrt(mean_squared_error(y_test_drop, y_pred_drop_xgb))
r2_drop_xgb = r2_score(y_test_drop, y_pred_drop_xgb)

print("\n=== Drop Model (XGBoost) ===")
print(f"RMSE: {rmse_drop_xgb:.4f}")
print(f"R2:   {r2_drop_xgb:.4f}")


Trained Drop-Specific XGBoost Model.
Predicted Drop-Specific XGBoost Model.

=== Drop Model (XGBoost) ===
RMSE: 3.0793
R2:   0.6422


## Step 3 (main step): Improve model

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb_drop_base = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

# Define the parameter grid
param_grid = {
    'learning_rate': [0.1, 0.05],      # 尝试正常和较小的学习率
    'max_depth': [5, 6, 7],             # 树的最大深度
    'n_estimators': [100, 200],         # 树的数量（配合学习率）
    'subsample': [0.8, 1.0],            # 采样比例
    'colsample_bytree': [0.8, 1.0]      # 特征采样比例
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_drop_base,
    param_grid=param_grid,
    scoring='r2',     
    cv=3,               
    verbose=1,
    n_jobs=-1          
)

# Fit GridSearchCV on the drop dataset
grid_search.fit(X_train_drop, y_train_drop)

# Best model
best_xgb_drop = grid_search.best_estimator_

# Print best params
print("Best parameters found:")
print(grid_search.best_params_)

# Predict and evaluate
y_pred_best_xgb = best_xgb_drop.predict(X_test_drop)

rmse_best = np.sqrt(mean_squared_error(y_test_drop, y_pred_best_xgb))
r2_best = r2_score(y_test_drop, y_pred_best_xgb)

print("\n=== Best Tuned XGBoost Drop Model ===")
print(f"RMSE: {rmse_best:.4f}")
print(f"R2:   {r2_best:.4f}")


Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best parameters found:
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}

=== Best Tuned XGBoost Drop Model ===
RMSE: 2.6688
R2:   0.7313


In [24]:

# Define new XGBoost Drop Model with updated parameters
drop_model_xgb_2 = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    learning_rate=0.1,
    max_depth=7,
    subsample=1.0,
    colsample_bytree=1.0,
    random_state=42,
    n_jobs=-1
)

# Train the model on the drop dataset
drop_model_xgb_2.fit(X_train_drop, y_train_drop)

print("Trained Updated XGBoost Drop Model (drop_model_xgb_2).")


Trained Updated XGBoost Drop Model (drop_model_xgb_2).


## Step 4: Predict on the test subset with y<0 

In [25]:
y_pred_drop_xgb_2 = drop_model_xgb_2.predict(X_test_drop)

# Evaluate XGBoost
rmse_drop_xgb_2 = np.sqrt(mean_squared_error(y_test_drop, y_pred_drop_xgb_2))
r2_drop_xgb_2 = r2_score(y_test_drop, y_pred_drop_xgb_2)

print("\n=== Drop Model (XGBoost) ===")
print(f"RMSE: {rmse_drop_xgb_2:.4f}")
print(f"R2:   {r2_drop_xgb_2:.4f}")



=== Drop Model (XGBoost) ===
RMSE: 2.7022
R2:   0.7245


## Step 5: Print the final result comparison summary table

In [26]:
# Predict using Original full model
y_pred_original_drop = xgb_wf.predict(X_test_drop)

# Evaluate original model
rmse_original_drop = np.sqrt(mean_squared_error(y_test_drop, y_pred_original_drop))
r2_original_drop = r2_score(y_test_drop, y_pred_original_drop)


print("\n=== Summary Comparison ===")
print(f"Original Full Model:    RMSE = {rmse_original_drop:.4f}, R2 = {r2_original_drop:.4f}")
print(f"Drop Model (XGBoost 1):    RMSE = {rmse_drop_xgb:.4f}, R2 = {r2_drop_xgb:.4f}")
print(f"Drop Model (new XGBoost 2):    RMSE = {rmse_drop_xgb_2:.4f}, R2 = {r2_drop_xgb_2:.4f}")



=== Summary Comparison ===
Original Full Model:    RMSE = 3.6740, R2 = 0.4907
Drop Model (XGBoost 1):    RMSE = 3.0793, R2 = 0.6422
Drop Model (new XGBoost 2):    RMSE = 2.7022, R2 = 0.7245
